Método de Broyden

In [4]:
import numpy as np
from sympy.abc import x
from sympy import *
import copy
import math as m


class GaussJordan:

    def intercambiarFilas(self, index1, index2, M): 
        M[index1],M[index2]=M[index2],M[index1]
        return M
   
    def multiplicarFila(self, k, fila, colInicial, M):
        M[fila] = k * M[fila]
        return M

    def restarFilas(self, f1, f2, M):
        M[f1] =  M[f2] - M[f1]
        return M 

    def buscarPivote(self, filas, col, M):
        indiceFila = -1
        maxNum = np.inf *-1
        for i in range(col+1, filas):
            if(M[i][col] > maxNum and M[i][col] != 0):
                indiceFila = i
                maxNum = M[i][col]
        return indiceFila

    def eliminacionGaussiana(self, f, c, M):
        # Definición de variables
        indicePiv = -1
        
        for i in range(f):
            pivote = M[i][i]
            if pivote == 0:
                indicePiv = self.buscarPivote(f, i, M) # Se implementa pivoteo parcial
                #TODO: Implementar pivoteo completo
                if indicePiv == -1:
                    print("El sistema no tiene solución")
                    exit(0)
                else:
                    M = self.intercambiarFilas(indicePiv, i, M)
                    pivote = M[i][i]
            
            for j in range(i+1, f): # Realizar la eliminación de los elementos debajo del pivote
                if M[j][i] != 0:
                    k = pivote / M[j][i]    # Multiplicador para la eliminación
                    M = self.multiplicarFila(k, j, i, M)
                    M = self.restarFilas(j, i, M)

    def GJ(self,f,c,M):
        for i in range(f):
            pivote=M[f-1-i][f-1-i]
            M[f-1-i]=M[f-1-i]/pivote
            for j in range(f-1-i):
              M[j]=M[j]-M[j][f-1-i]*M[f-1-i]

    def inversa(self,f,M):
      I=np.identity(f)
      matriz=np.concatenate((M,I),axis=1)
      return matriz

    def obtenerInversa(self,f,matriz):
      inversa=[]
      for i in range(f):
        aux=[]
        for j in range(f):
          aux.append(matriz[i][j+len(matriz)])
        inversa.append(aux)
      return inversa

class BroydenMethod:

  def norma(self,vector0,vector1):
    # Cálculo de error por definición de norma
    suma=0
    for i in range(len(vector0)):
      aux=(vector0[i]-vector1[i])**2
      suma=suma+aux
    return m.sqrt(suma)


  def evaluateF(self,xs,initPoint,F):
    # Evaluar en F(x)
    F1=copy.copy(F)
    for i in range(len(F1)):
      for j in range(len(initPoint)):
        F1[i]=F1[i].subs(xs[j],initPoint[j])
    return F1

  def jacobian(self,xs,initPoint,F):
    J=[] # Creación de Jacobiana
    for i in range(len(xs)):
      aux=[]
      for j in range(len(xs)):
        aux.append(diff(F[i],xs[j]))
      J.append(aux)

    # Evaluar en Jacobiana
    for i in range(len(J)):
      for j in range(len(J)):
        for k in range(len(initPoint)):
          J[i][j]=J[i][j].subs(xs[k],initPoint[k])

    return J
  
  def broyden(self,xs,X0,F):
    # BroydenMethod
    objB=BroydenMethod()
    F1=objB.evaluateF(xs,X0,F)

    J=objB.jacobian(xs,X0,F)

    objGJ=GaussJordan()
    matriz=objGJ.inversa(len(J),J)
    objGJ.eliminacionGaussiana(len(J),len(J[0]),matriz)
    objGJ.GJ(len(J),len(J[0]),matriz)
    A0=objGJ.obtenerInversa(len(J),matriz)

    # Primer valor de Newton
    X1=X0-np.dot(A0,F1)
    print("iteracion 0","Solución:",X1)

    # Broyden y comienza la segunda iteración
    tol=0.000001
    maxIter=100
    iteracion=1
    error=np.inf

    while (error>tol and iteracion<maxIter):
      Y1=np.transpose(np.matrix(objB.evaluateF(xs,X1,F))-np.matrix(objB.evaluateF(xs,X0,F)))
      S1=np.transpose(np.matrix(X1-X0))

      denominador=float(np.dot(np.transpose(S1),np.dot(A0,Y1)))

      aux1=S1-np.dot(A0,Y1)
      aux2=np.dot(np.transpose(S1),A0)
      numerador=np.dot(aux1,aux2)

      fraccion=(1/denominador)*numerador
      A1=np.array(A0+fraccion)

      # Regresando a Newton
      X2=X1-np.dot(A1,objB.evaluateF(xs,X1,F))
      error=objB.norma(X2,X1)
      print("iteracion",iteracion,"Solución:",X2)
      
      # Regresando a valores
      iteracion=iteracion+1
      X0=X1
      X1=X2
      A0=A1

    return X2


def main():
  n=3 # Número de ecuaciones y variables
  xs=[] # Incógnitas
  for i in range(n):
    xs.append(symbols("x"+str(i+1)))

  f1=3*xs[0] - cos(xs[1]*xs[2]) - 1/2
  f2=xs[0]**2 - 81*(xs[1] + 0.1)**2 + sin(xs[2]) + 1.06
  f3=np.e**(-xs[0]*xs[1]) + 20*xs[2] + ((10 * np.pi) -3) /3
  #f1=xs[0]**2-xs[1]**2-1
  #f2=xs[0]**2+xs[1]**2+xs[0]*xs[1]-4

  F=[]
  F.append(f1)
  F.append(f2)
  F.append(f3)
  #X0=[1,1]
  X0=[0.1,0.1,-0.1]
  print("Punto inicial:",X0)

  objBM=BroydenMethod()
  aproximation=objBM.broyden(xs,X0,F)
  print("Aproximación final:",aproximation)

if __name__=="__main__":
  main()

Punto inicial: [0.1, 0.1, -0.1]
iteracion 0 Solución: [0.499869672926428 0.0194668485374181 -0.521520471935831]
iteracion 1 Solución: [0.499986375456912 0.00873783929925742 -0.523174574399749]
iteracion 2 Solución: [0.500006597059974 0.000867273555790280 -0.523572341486402]
iteracion 3 Solución: [0.500000328717546 3.95282753059880e-5 -0.523597685378835]
iteracion 4 Solución: [0.500000001566878 1.93543975108938e-7 -0.523598770059983]
iteracion 5 Solución: [0.500000000000334 5.34641261305872e-13 -0.523598775599102]
Aproximación final: [0.500000000000334 5.34641261305872e-13 -0.523598775599102]
